In [1]:
import os, sys, time
import numpy as np
import pydicom as dcm

In [236]:
PATH = '/mnt/iDriveShare/Kayla/CBCT_images/kayla_extracted/'

In [229]:
file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
file_list.sort()

In [230]:
list_models = []
model_dict = {'Patient Verification':'CBCT', 'Brilliance Big Bore':'CT', 'ARIA RadOnc':'RT'}

def basic_sort_files_by_date(list_images):
    for file in list_images:
        d = dcm.read_file(PATH + file)
        man_model = d.ManufacturerModelName
        
        try:
            dicom_type = model_dict[man_model]
        except:
            dicom_type = man_model
            
        if man_model not in list_models:
            list_models.append(man_model)
        
        try: 
            new_path = PATH + d.AcquisitionDate
        except:
            new_path = PATH + d.StudyDate
            
        if not os.path.exists(new_path):
            os.system("sudo mkdir " + new_path)
            print("Created Directory" + new_path)
        os.system("sudo mv " + PATH+file +" " + new_path+"/"+file)
        
        file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]

In [237]:
def find_RT_dicoms(file_list):
    list_RT = []
    list_RE = []
    list_RS = []
    list_RI = []
    list_RD = []
    other = []

    for file in file_list:
        if 'RT' in file:
            list_RT.append(file)
        elif 'RE' in file:
            list_RE.append(file)
        elif 'RS' in file:
             list_RS.append(file)
        elif 'RI' in file:
            list_RI.append(file)
        elif 'RD' in file:
            list_RD.append(file)
        elif 'CT' not in file:
            other.append(file)
            
    print("RT: " , len(list_RT))
    print("RS: " , len(list_RS))
    print("RI: " , len(list_RI))
    print("RE: ", len(list_RE))
    print("RD: ", len(list_RD))
    print("other: " , len(other))
        

In [83]:
dict_class_UID = {'1.2.840.10008.5.1.4.1.1.2': 'CT', '1.2.840.10008.5.1.4.1.1.481.1': 'RI' }

In [235]:
def remove_RI_RT_files(file_list):
    RI_count = 0
    RT_count = 0
    RE_count = 0
    
    RT_path = PATH + "RT"
    if not os.path.exists(RT_path):
        os.system("sudo mkdir " + RT_path)
        print("Created directory "+RT_path)

    RI_path = PATH + "RI"
    if not os.path.exists(RI_path):
        os.system("sudo mkdir " + RI_path)
        print("Created directory "+RI_path)
    
    for file in file_list:
        if 'RT' in file:
            os.system("sudo mv " + PATH+file +" " + RT_path+"/"+file)
            RT_count += 1
        elif 'RI' in file:
            os.system("sudo mv " + PATH+file +" " + RI_path+"/"+file)
            RI_count += 1
        elif 'RE' in file:
            RE_class = dcm.read_file(PATH+file).ReferencedSeriesSequence[0].ReferencedInstanceSequence[0].ReferencedSOPClassUID
            if dict_class_UID[RE_class] == 'RI':
                os.system("sudo mv " + PATH+file +" " + RI_path+"/"+file)
                RE_count += 1
    print("----------------------------------------")
    print("Files moved: ", RT_count, " RT, ", RI_count, " RI, ",RE_count, " RE")
    file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
    print("Files remaining: ", len(file_list))
                
                
            

In [234]:
def sort_image_files_by_RS(file_list):
    uid_dict = {}
    list_RE = []
    list_RS = []
    list_RD = []
    CT_count = 0
    other = []
    
    for file in file_list:
        if 'RE' in file:
            list_RE.append(file)
        elif 'RS' in file:
             list_RS.append(file)
        elif 'RD' in file:
            list_RD.append(file)
        elif 'CT' not in file:
            other.append(file)
            
    for file in list_RS:
        if 'RS' in file:
            d = dcm.read_file(PATH+file)
            
            new_path = PATH + d.StructureSetLabel
#             print(new_path)
            if not os.path.exists(new_path):
                os.system("sudo mkdir " + new_path)
                print("Created directory "+new_path)
        
            os.system("sudo mv " + PATH+file +" " + new_path+"/"+file)
    
            frame_of_reference_uid = d.ReferencedFrameOfReferenceSequence[0].FrameOfReferenceUID
            uid_dict.update({frame_of_reference_uid: new_path})
        
            
            for img in d.ReferencedFrameOfReferenceSequence[0].RTReferencedStudySequence[0].RTReferencedSeriesSequence[0].ContourImageSequence:
                uid = img.ReferencedSOPInstanceUID 
                os.system("sudo mv " + PATH+"CT."+uid+".dcm" +" " + new_path+"/"+"CT."+uid+".dcm")
                CT_count += 1

    
    for file in list_RE:
        d = dcm.read_file(PATH+file)
        try:
            frame_of_reference_uid = d.RegistrationSequence[1].FrameOfReferenceUID
        except:
            print("HERE")
            frame_of_reference_uid = d.FrameOfReferenceUID
        os.system("sudo mv " + PATH+file +" " + uid_dict[frame_of_reference_uid]+"/"+file)

    for file in list_RD:
        d = dcm.read_file(PATH+file)
        frame_of_reference_uid = d.FrameOfReferenceUID
        os.system("sudo mv " + PATH+file +" " + uid_dict[frame_of_reference_uid]+"/"+file)
    
    if len(other) != 0:
        print("Other files not moved")
        for file in other:
            print(file)
    
    print("----------------------------------------")
    print("Files moved: ",CT_count, " CT, ", len(list_RS), " RS, ", len(list_RE), " RE, ",len(list_RD), " RD")
    file_list = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
    print("Files remaining: ", len(file_list))

In [ ]:
find_RT_dicoms(file_list)

In [ ]:
remove_RI_RT_files(file_list)

In [189]:
sort_image_files_by_RS(file_list)

KeyError: '1.2.246.352.221.516817255461858266617223958142835396797'